# Capstone 1 (from Chapter 5) — Monte Carlo Estimation of Pi

In [1]:
!uv pip install numpy

Audited 1 package in 0.55ms


In [2]:
LOGGING_ENABLED = True

In [3]:
import logging

from llm_agents_from_scratch.logger import enable_console_logging

if LOGGING_ENABLED:
    enable_console_logging(logging.INFO)

## Build Tools

In [4]:
import uuid

import numpy as np
from pydantic import BaseModel, Field, computed_field

from llm_agents_from_scratch.tools import PydanticFunctionTool

### Tool: `generate_random_sample()`

In [5]:
# Global registry to store samples
SAMPLE_REGISTRY: dict[str, list[tuple[float, float]]] = {}

In [6]:
class RandomSampleParams(BaseModel):
    """Params for generate_random_sample."""

    n: int = Field(description="The number of random points to generate")


class RandomSample(BaseModel):
    """Result from generate_random_sample."""

    sample_id: str = Field(
        description="Pass this sample_id to monte_carlo_estimate",
    )

    @computed_field
    @property
    def sample_size(
        self,
    ) -> int:
        """Determine n from SAMPLE_REGISTRY."""
        return len(SAMPLE_REGISTRY[self.sample_id])


def generate_random_sample(params: RandomSampleParams) -> RandomSample:
    """Generate n random points in [-1, 1] × [-1, 1].

    Returns a sample_id. Pass this sample_id directly to monte_carlo_estimate.
    """
    orig_pts = np.random.uniform(size=(params.n, 2))
    transformed = 1 - 2 * (1 - orig_pts)

    sample_id = str(uuid.uuid4())
    SAMPLE_REGISTRY[sample_id] = [tuple(pt) for pt in transformed.tolist()]

    return RandomSample(sample_id=sample_id)

In [7]:
# test generate_random_sample() function
rs = generate_random_sample(RandomSampleParams(n=1000))
rs.model_dump()

{'sample_id': '3f6a213f-770d-41b5-8cfa-f8ba7a3b866c', 'sample_size': 1000}

In [8]:
# create tool
random_sample_tool = PydanticFunctionTool(generate_random_sample)
random_sample_tool.parameters_json_schema

{'description': 'Params for generate_random_sample.',
 'properties': {'n': {'description': 'The number of random points to generate',
   'title': 'N',
   'type': 'integer'}},
 'required': ['n'],
 'title': 'RandomSampleParams',
 'type': 'object'}

### Tool: `add_more_points()`

In [9]:
class AddPointsParams(BaseModel):
    """Params for generate_random_sample."""

    sample_id: str = Field(
        description="The sample_id of the sample to augment",
    )
    n: int = Field(description="The number of random points to generate")


def add_more_points_to_sample(params: AddPointsParams) -> RandomSample:
    """Add n more random points to an existing random sample.

    Returns a sample_id and the total number of points.
    """
    orig_pts = np.random.uniform(size=(params.n, 2))
    transformed = 1 - 2 * (1 - orig_pts)

    # augment sample
    SAMPLE_REGISTRY[params.sample_id] += [
        tuple(pt) for pt in transformed.tolist()
    ]

    return RandomSample(sample_id=params.sample_id)

In [10]:
# test add_more_points_to_sample() function
rs = add_more_points_to_sample(AddPointsParams(n=1000, sample_id=rs.sample_id))
str(rs)

"sample_id='3f6a213f-770d-41b5-8cfa-f8ba7a3b866c' sample_size=2000"

In [11]:
# create tool
add_more_points_tool = PydanticFunctionTool(add_more_points_to_sample)
add_more_points_tool.parameters_json_schema

{'description': 'Params for generate_random_sample.',
 'properties': {'sample_id': {'description': 'The sample_id of the sample to augment',
   'title': 'Sample Id',
   'type': 'string'},
  'n': {'description': 'The number of random points to generate',
   'title': 'N',
   'type': 'integer'}},
 'required': ['sample_id', 'n'],
 'title': 'AddPointsParams',
 'type': 'object'}

### Tool: `monte_carlo_estimate()`

In [12]:
class MonteCarloEstimateParams(BaseModel):
    """Params for monte_carlo_estimate."""

    sample_id: str = Field(
        description="The sample_id returned by generate_random_sample",
    )


class MonteCarloEstimateResults(BaseModel):
    """Params for monte_carlo_estimate."""

    sample_id: str
    sample_size: int
    estimate: float


def monte_carlo_estimate(
    params: MonteCarloEstimateParams,
) -> MonteCarloEstimateResults:
    """Estimate pi using Monte Carlo method.

    Args:
        params: Contains sample_id from generate_random_sample.

    Returns:
        Estimate of pi (float).
    """
    points = SAMPLE_REGISTRY[params.sample_id]
    n = len(points)
    inside = sum((x**2 + y**2) < 1 for x, y in points)
    return MonteCarloEstimateResults(
        estimate=(inside / n) * 4,
        sample_id=params.sample_id,
        sample_size=n,
    )

In [13]:
pi_estimate = monte_carlo_estimate(
    MonteCarloEstimateParams(sample_id=rs.sample_id),
)
pi_estimate

MonteCarloEstimateResults(sample_id='3f6a213f-770d-41b5-8cfa-f8ba7a3b866c', sample_size=2000, estimate=3.168)

In [14]:
monte_carlo_estimate_tool = PydanticFunctionTool(monte_carlo_estimate)
monte_carlo_estimate_tool.parameters_json_schema

{'description': 'Params for monte_carlo_estimate.',
 'properties': {'sample_id': {'description': 'The sample_id returned by generate_random_sample',
   'title': 'Sample Id',
   'type': 'string'}},
 'required': ['sample_id'],
 'title': 'MonteCarloEstimateParams',
 'type': 'object'}

## Define our LLMAgent

In [15]:
from llm_agents_from_scratch import LLMAgent
from llm_agents_from_scratch.llms import OllamaLLM

model = "qwen3:8b"
llm = OllamaLLM(model)
llm_agent = LLMAgent(
    llm=llm,
    tools=[
        random_sample_tool,
        add_more_points_tool,
        monte_carlo_estimate_tool,
    ],
)

## Define the Task

In [16]:
from llm_agents_from_scratch.data_structures import Task

In [17]:
instruction_template = """
You are given tools to estimate pi using Monte Carlo methods.

Your target: Estimate pi accurate to 4 decimal places (3.1415).

<tools>
1. `generate_random_sample(n)`
        → Creates new sample, returns sample_id and sample_size
2. `add_more_points_to_sample(sample_id, n)`
        → Adds n points, returns updated sample_size
3. `monte_carlo_estimate(sample_id)`
        → Returns pi estimate (float)
</tools>

<workflow>
1. generate_random_sample(n=1000) → get sample_id
2. monte_carlo_estimate(sample_id) → get estimate
3. Check: does round(estimate, 4) == 3.1415?
   - YES → Report success and stop
   - NO → MUST call add_more_points_to_sample(sample_id, n=5000)
        to improve precision
4. After adding points, call monte_carlo_estimate again
5. Repeat steps 3-4 until converged
</workflow>

<warnings>
NEVER fabricate tool results—PI estimates should only come from
    monte_carlo_estimate tool.
NEVER make multiple tool calls in one response.
NEVER continue after a tool call - end your response immediately.
ALWAYS wait for the actual tool response before proceeding.
</warnings>
""".strip()

In [18]:
task = Task(
    instruction=instruction_template,
)

## Perform the Task

In [19]:
handler = llm_agent.run(task, max_steps=10)

INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: You are given tools to estimate pi using Monte Carlo methods.

Your target: Estimate pi accurate to 4 decimal places (3.1415).

<tool...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      ⚙️ Processing Step: You are given tools to estimate pi using Monte Carlo methods.

Your target: Estimate pi accurate to 4 decimal places (3.1415).

<t...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      🛠️ Executing Tool Call: generate_random_sample
INFO (llm_agents_fs.TaskHandler) :      ✅ Successful Tool Call: sample_id='f029286f-6b3b-4382-86b9-a81c8e163747' sample_size=1000
INFO (llm_agents_fs.TaskHandler) :      ✅ Step Result: I need to call the monte_carlo_estimate tool with sample_id 'f029286f-6b3b-4382-86b9-a81c8e163747' to get the initial pi estimate.
INFO (llm_agents_fs.TaskHandler) :      🧠 New Step: Call the `monte_carlo_estimate` tool with sample_id 'f029286f-6b3b-4382-86b9-a81c8e163747' to obtain the initial pi estimate.
INFO (llm_ag

In [20]:
handler.done()

True

In [21]:
print(handler.rollout)

=== Task Step Start ===

💬 assistant: The current instruction is 'You are given tools to estimate pi using Monte Carlo methods.

Your target: Estimate pi accurate to 4 decimal places (3.1415).

<tools>
1. `generate_random_sample(n)` → Creates new sample, returns sample_id and sample_size
2. `add_more_points_to_sample(sample_id, n)` → Adds n points, returns updated sample_size
3. `monte_carlo_estimate(sample_id)` → Returns pi estimate (float)
</tools>

<workflow>
1. generate_random_sample(n=1000) → get sample_id
2. monte_carlo_estimate(sample_id) → get estimate
3. Check: does round(estimate, 4) == 3.1415?
   - YES → Report success and stop
   - NO → MUST call add_more_points_to_sample(sample_id, n=5000) to improve precision
4. After adding points, call monte_carlo_estimate again
5. Repeat steps 3-4 until converged
</workflow>

<warnings>
NEVER fabricate tool results—PI estimates should only come from
    monte_carlo_estimate tool.
NEVER make multiple tool calls in one response.
NEVER co

In [22]:
result = (
    handler.result() if not handler.exception() else str(handler.exception())
)
print(result)

Max steps reached.
